In [6]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
import scipy.stats
from numpy.linalg import inv
import pandas_datareader as pdr

In [7]:
# OLD DATA LOADING
"""

data = pd.read_csv("data/25_Portfolios_5x5.csv",skiprows=15,index_col="DATE",parse_dates=True,nrows=1134)
data = data.dropna()
data.index = pd.to_datetime(data.index,format="%Y%m")

factors = pd.read_csv("data/F-F_Research_Data_Factors.csv",skiprows=3,index_col=0,parse_dates=True,nrows=1134)
factors = factors.dropna()
factors.index = pd.to_datetime(factors.index,format="%Y%m")
rm = factors["Mkt-RF"]
rf = factors["RF"]

mom = pd.read_csv("data/F-F_Momentum_Factor.csv",skiprows=13,index_col=0,parse_dates=True,nrows=1129)
mom = mom.dropna()
mom.index = pd.to_datetime(mom.index,format="%Y%m")

reversal = pd.read_csv("data/F-F_ST_Reversal_Factor.csv",skiprows=13,index_col=0,parse_dates=True,nrows=1139)
reversal = reversal.dropna()
reversal.index = pd.to_datetime(reversal.index,format="%Y%m")

factors_5 = pd.read_csv("data/F-F_Research_Data_5_Factors_2x3.csv",skiprows=3,index_col=0,parse_dates=True,nrows=690)
factors_5 = factors_5.dropna()
factors_5.index = pd.to_datetime(factors_5.index,format="%Y%m")

data = data["1963-07-01":"2020-06-01"]
rm = rm["1963-07-01":"2020-06-01"]
rf = rf["1963-07-01":"2020-06-01"]
factors_5 = factors_5["1963-07-01":"2020-06-01"]
factors = factors["1963-07-01":"2020-06-01"]
mom = mom["1963-07-01":"2020-06-01"]
reversal = reversal["1963-07-01":"2020-06-01"]

ex_data = data.subtract(rf,axis=0)
factors_5 = factors_5.drop("RF",axis=1)
factors_3 = factors.drop("RF",axis=1)
factors_3_mom = factors_3.copy()
factors_3_mom["Mom"] = mom
factors_3_rev = factors_3.copy()
factors_3_rev["ST Rev"] = reversal
"""

'\n\ndata = pd.read_csv("data/25_Portfolios_5x5.csv",skiprows=15,index_col="DATE",parse_dates=True,nrows=1134)\ndata = data.dropna()\ndata.index = pd.to_datetime(data.index,format="%Y%m")\n\nfactors = pd.read_csv("data/F-F_Research_Data_Factors.csv",skiprows=3,index_col=0,parse_dates=True,nrows=1134)\nfactors = factors.dropna()\nfactors.index = pd.to_datetime(factors.index,format="%Y%m")\nrm = factors["Mkt-RF"]\nrf = factors["RF"]\n\nmom = pd.read_csv("data/F-F_Momentum_Factor.csv",skiprows=13,index_col=0,parse_dates=True,nrows=1129)\nmom = mom.dropna()\nmom.index = pd.to_datetime(mom.index,format="%Y%m")\n\nreversal = pd.read_csv("data/F-F_ST_Reversal_Factor.csv",skiprows=13,index_col=0,parse_dates=True,nrows=1139)\nreversal = reversal.dropna()\nreversal.index = pd.to_datetime(reversal.index,format="%Y%m")\n\nfactors_5 = pd.read_csv("data/F-F_Research_Data_5_Factors_2x3.csv",skiprows=3,index_col=0,parse_dates=True,nrows=690)\nfactors_5 = factors_5.dropna()\nfactors_5.index = pd.to_dat

In [8]:
# Load required data
reversal = pdr.get_data_famafrench("F-F_ST_Reversal_Factor", start="1963-07", end="2020-06")[0] #* (1/100)
mom = pdr.get_data_famafrench("F-F_Momentum_Factor", start="1963-07", end="2020-06")[0] #* (1/100)
factors = pdr.get_data_famafrench("F-F_Research_Data_Factors", start="1963-07", end="2020-06")[0] #* (1/100)
factors_5 = pdr.get_data_famafrench("F-F_Research_Data_5_Factors_2x3", start="1963-07", end="2020-06")[0] #* (1/100)
data = pdr.get_data_famafrench("25_Portfolios_5x5", start="1963-07", end="2020-06")[0] #* (1/100)

# Portfolio returns must be excess for GRS
ex_data = data.sub(factors["RF"], axis=0)

# Remove RF from dataframes
factors_5 = factors_5.drop(columns=["RF"])
factors = factors.drop(columns=["RF"])

#For 5 Factors
#ff_5_factors_constant = sm.add_constant(ff_5_factors)

# For 3 Factors + Momentum
factors_3_mom = pd.concat([factors["Mkt-RF"], factors["SMB"], factors["HML"], mom], axis=1)
#factors_3_mom = sm.add_constant(factors_3_mom)

# For 3 Factors + reversal
factors_3_rev = pd.concat([factors["Mkt-RF"], factors["SMB"], factors["HML"], reversal], axis=1)
#factors_3_rev = sm.add_constant(factors_3_rev)

In [9]:
factors_5

,Mkt-RF,SMB,HML,RMW,CMA
Date,,,,,
1963-07,-0.39,-0.45,-0.94,0.66,-1.15
1963-08,5.07,-0.82,1.82,0.40,-0.40
1963-09,-1.57,-0.48,0.17,-0.76,0.24
1963-10,2.53,-1.30,-0.04,2.75,-2.24
1963-11,-0.85,-0.85,1.70,-0.45,2.22
...,...,...,...,...,...
2020-02,-8.13,-0.03,-3.92,-1.61,-2.49
2020-03,-13.38,-8.38,-13.96,-1.38,1.21
2020-04,13.65,2.79,-1.39,2.51,-1.03


In [83]:
alphas_5 = []
e5 = []


for columns in ex_data:
    x = sm.add_constant(factors_5)
    res = sm.OLS(ex_data[columns],x).fit()
    alphas_5.append(res.params[0])
    e5.append(ex_data[columns] - res.predict())
pd.DataFrame(np.array(e5) @ np.array(e5).T)
e5_cov = (pd.DataFrame(e5).T).cov() 

In [89]:
alphas_5 = []
alphas_mom = []
alphas_rev = []
e5 = []
emom = []
erev = []

for columns in ex_data:
    x = sm.add_constant(factors_5)
    res = sm.OLS(ex_data[columns],x).fit()
    alphas_5.append(res.params[0])
    e5.append(ex_data[columns] - res.predict())

    x = sm.add_constant(factors_3_mom)
    res = sm.OLS(ex_data[columns],x).fit()
    alphas_mom.append(res.params[0])
    emom.append(ex_data[columns] - res.predict())

    x = sm.add_constant(factors_3_rev)
    res = sm.OLS(ex_data[columns],x).fit()
    alphas_rev.append(res.params[0])
    erev.append(ex_data[columns] - res.predict())

e5_cov = (pd.DataFrame(e5).T).cov() 
emom_cov = (pd.DataFrame(emom).T).cov() 
erev_cov = (pd.DataFrame(erev).T).cov() 

In [90]:
def calc_GRS(alpha_vector, factor_dataframe, asset_returns,e_cov):
    gen_cov = factor_dataframe.cov()
    generalized_sharpe = factor_dataframe.mean().T @ inv(gen_cov) @ factor_dataframe.mean()     #Cochrane 2005, p.217
    #cov = asset_returns.cov()
    cov = e_cov
    if isinstance(alpha_vector, list):
        alpha_vector = np.array(alpha_vector)
    #Calc GRS
    df = len(asset_returns) - len(asset_returns.columns) - len(factor_dataframe.columns) # T - N - K     (N = number of assets, K = number of factors)
    N = len(asset_returns.columns)
    GRS = (df/ N) * ((alpha_vector.T @ inv(cov) @ alpha_vector) / (1+ generalized_sharpe))
    p = 1-scipy.stats.f.cdf(GRS, len(asset_returns.columns), df) #find p-value of F test statistic
    return GRS, p

In [91]:
calc_GRS(alphas_5,factors_5,data,e5_cov)

(3.037721469314005, 1.3217940193488431e-06)

In [92]:
calc_GRS(alphas_mom,factors_3_mom,data,emom_cov)

(2.9797826379846293, 2.08801834578054e-06)

In [93]:
calc_GRS(alphas_rev,factors_3_rev, data,erev_cov)

(3.7454974738078146, 4.108721696205464e-09)

In [15]:
## Farma MacBeth methodology for 5 factors

beta_mkt = []
beta_smb = []
beta_hml = []
beta_rmw = []
beta_cma = []

for columns in ex_data:
    x = sm.add_constant(factors_5)
    res = sm.OLS(ex_data[columns],x).fit()
    beta_mkt.append(res.params[1])
    beta_smb.append(res.params[2])
    beta_hml.append(res.params[3])
    beta_rmw.append(res.params[4])
    beta_cma.append(res.params[5])

betas = pd.DataFrame([beta_mkt,beta_smb,beta_hml,beta_rmw,beta_cma]).T
betas.columns = ["Mkt-RF","SMB","HML","RMW","CMA"]
betas = sm.add_constant(betas)
betas.index = ex_data.T.index

alpha = []
lambda_mkt = []
lambda_SMB = []
lambda_HML = []
lambda_RMW = []
lambda_CMA = []

for column in ex_data.T:
    x = betas
    y = ex_data.T[column]
    res = sm.OLS(y,x).fit()
    alpha.append(res.params[0])
    lambda_mkt.append(res.params[1])
    lambda_SMB.append(res.params[2])
    lambda_HML.append(res.params[3])
    lambda_RMW.append(res.params[4])
    lambda_CMA.append(res.params[5])

alpha = pd.DataFrame(np.array(alpha))
lambda_mkt = pd.DataFrame(np.array(lambda_mkt))
lambda_SMB = pd.DataFrame(np.array(lambda_SMB))
lambda_HML = pd.DataFrame(np.array(lambda_HML))
lambda_RMW = pd.DataFrame(np.array(lambda_RMW))
lambda_CMA = pd.DataFrame(np.array(lambda_CMA))

lambdas = pd.concat([lambda_mkt, lambda_SMB, lambda_HML, lambda_RMW, lambda_CMA], axis=1)
lambdas.columns = ['lambda_mkt', "lambda_SMB", 'lambda_HML', "lambda_RMW", 'lambda_CMA']

params = [alpha, lambda_mkt, lambda_SMB, lambda_HML, lambda_RMW, lambda_CMA]

variances = []
for i in params:
    deviations = i - i.mean()
    sum_of_squared_dev = (deviations ** 2).sum()
    var = sum_of_squared_dev / (len(i) ** 2) 
    variances.append(var)


variances = pd.DataFrame(np.array(variances)).T
variances.columns = ['alpha', 'lambda_mkt', 'lambda_SMB', 'lambda_HML', 'lambda_RMW', 'lambda_CMA']

#Shanken Adjustment
adjustment = (1 + (lambdas.mean().T @ inv(factors_5.cov()) @ lambdas.mean()))
#Adjusted variances
variances = variances * adjustment

#t-tests
t_alpha = alpha.mean() / np.sqrt(variances['alpha'])
t_mkt = lambda_mkt.mean() / np.sqrt(variances['lambda_mkt'])
t_SMB = lambda_SMB.mean() / np.sqrt(variances['lambda_SMB'])
t_HML = lambda_HML.mean() / np.sqrt(variances['lambda_HML'])
t_RMW = lambda_RMW.mean() / np.sqrt(variances['lambda_RMW'])
t_CMA = lambda_CMA.mean() / np.sqrt(variances['lambda_CMA'])
t_stats = pd.concat([t_alpha, t_mkt, t_SMB, t_HML, t_RMW, t_CMA], axis=1)
t_stats.columns = ['t_alpha', 't_mkt', 't_SMB', 't_HML', 't_RMW', 't_CMA']
t_stats = np.round(t_stats,2)

print("Factor premia and pricing error for the 5 factor model are:")
print()
print("Pricing error:", alpha.mean())
print()
print("Factor Premia:", np.round(lambdas.mean(),4))
print()
print(t_stats)


Factor premia and pricing error for the 5 factor model are:

Pricing error: 0    0.939512
dtype: float64

Factor Premia: lambda_mkt   -0.4225
lambda_SMB    0.2639
lambda_HML    0.2223
lambda_RMW    0.4358
lambda_CMA    0.0186
dtype: float64

   t_alpha  t_mkt  t_SMB  t_HML  t_RMW  t_CMA
0     3.38  -1.28   2.15    1.9   2.54   0.11


In [16]:
## Farma MacBeth methodology for 3 factors and momentum

beta_mkt = []
beta_smb = []
beta_hml = []
beta_mom = []

for columns in ex_data:
    x = sm.add_constant(factors_3_mom)
    res = sm.OLS(ex_data[columns],x).fit()
    beta_mkt.append(res.params[1])
    beta_smb.append(res.params[2])
    beta_hml.append(res.params[3])
    beta_mom.append(res.params[4])

betas = pd.DataFrame([beta_mkt,beta_smb,beta_hml,beta_mom]).T
betas.columns = ["Mkt-RF","SMB","HML","Mom"]
betas = sm.add_constant(betas)
betas.index = ex_data.T.index

alpha = []
lambda_mkt = []
lambda_SMB = []
lambda_HML = []
lambda_Mom = []

for column in ex_data.T:
    x = betas
    y = ex_data.T[column]
    res = sm.OLS(y,x).fit()
    alpha.append(res.params[0])
    lambda_mkt.append(res.params[1])
    lambda_SMB.append(res.params[2])
    lambda_HML.append(res.params[3])
    lambda_Mom.append(res.params[4])

alpha = pd.DataFrame(np.array(alpha))
lambda_mkt = pd.DataFrame(np.array(lambda_mkt))
lambda_SMB = pd.DataFrame(np.array(lambda_SMB))
lambda_HML = pd.DataFrame(np.array(lambda_HML))
lambda_Mom = pd.DataFrame(np.array(lambda_Mom))

lambdas = pd.concat([lambda_mkt, lambda_SMB, lambda_HML, lambda_Mom], axis=1)
lambdas.columns = ['lambda_mkt', "lambda_SMB", 'lambda_HML', "lambda_Mom"]

params = [alpha, lambda_mkt, lambda_SMB, lambda_HML, lambda_Mom]


variances = []

for i in params:
    deviations = i - i.mean(axis=0)
    sum_of_squared_dev = (deviations ** 2).sum()
    var = sum_of_squared_dev / (len(i) ** 2) 
    variances.append(var)

variances = pd.DataFrame(np.array(variances)).T
variances.columns = ['alpha', 'lambda_mkt', 'lambda_SMB', 'lambda_HML', 'lambda_Mom']

"""
for i in params:
    var = ((i.sub(i.mean(), axis=1)) ** 2).sum() / (len(i) ** 2)
    variances.append(var)

variances = pd.DataFrame(np.array(variances)).T
variances.columns = ['alpha', 'lambda_mkt', 'lambda_SMB', 'lambda_HML', 'lambda_Mom']
"""


#Shanken Adjustment
adjustment = (1 + (lambdas.mean().T @ inv(factors_3_mom.cov()) @ lambdas.mean()))
#Adjusted variances
variances = variances * adjustment

#t-tests
t_alpha = alpha.mean() / np.sqrt(variances['alpha'])
t_mkt = lambda_mkt.mean() / np.sqrt(variances['lambda_mkt'])
t_SMB = lambda_SMB.mean() / np.sqrt(variances['lambda_SMB'])
t_HML = lambda_HML.mean() / np.sqrt(variances['lambda_HML'])
t_Mom = lambda_Mom.mean() / np.sqrt(variances['lambda_Mom'])
t_stats = pd.concat([t_alpha, t_mkt, t_SMB, t_HML, t_Mom], axis=1)
t_stats.columns = ['t_alpha', 't_mkt', 't_SMB', 't_HML', 't_Mom']
t_stats = np.round(t_stats,2)

print("Factor premia and pricing error for the 3 factor and momentum model are:")
print()
print("Pricing error:", alpha.mean())
print()
print("Factor Premia:", np.round(lambdas.mean(),4))
print()
print(t_stats)


Factor premia and pricing error for the 3 factor and momentum model are:

Pricing error: 0    0.403892
dtype: float64

Factor Premia: lambda_mkt    0.1996
lambda_SMB    0.1452
lambda_HML    0.3113
lambda_Mom    2.4873
dtype: float64

   t_alpha  t_mkt  t_SMB  t_HML  t_Mom
0     1.11   0.47   1.01   2.29   2.98


In [17]:
## Farma MacBeth methodology for 3 factors and ST reversal factor

beta_mkt = []
beta_smb = []
beta_hml = []
beta_rev = []

for columns in ex_data:
    x = sm.add_constant(factors_3_rev)
    res = sm.OLS(ex_data[columns],x).fit()
    beta_mkt.append(res.params[1])
    beta_smb.append(res.params[2])
    beta_hml.append(res.params[3])
    beta_rev.append(res.params[4])

betas = pd.DataFrame([beta_mkt,beta_smb,beta_hml,beta_rev]).T
betas.columns = ["Mkt-RF","SMB","HML","REV"]
betas = sm.add_constant(betas)
betas.index = ex_data.T.index

alpha = []
lambda_mkt = []
lambda_SMB = []
lambda_HML = []
lambda_REV = []

for column in ex_data.T:
    x = betas
    y = ex_data.T[column]
    res = sm.OLS(y,x).fit()
    alpha.append(res.params[0])
    lambda_mkt.append(res.params[1])
    lambda_SMB.append(res.params[2])
    lambda_HML.append(res.params[3])
    lambda_REV.append(res.params[4])

alpha = pd.DataFrame(np.array(alpha))
lambda_mkt = pd.DataFrame(np.array(lambda_mkt))
lambda_SMB = pd.DataFrame(np.array(lambda_SMB))
lambda_HML = pd.DataFrame(np.array(lambda_HML))
lambda_REV = pd.DataFrame(np.array(lambda_REV))

lambdas = pd.concat([lambda_mkt, lambda_SMB, lambda_HML, lambda_REV], axis=1)
lambdas.columns = ['lambda_mkt', "lambda_SMB", 'lambda_HML', "lambda_REV"]

params = [alpha, lambda_mkt, lambda_SMB, lambda_HML, lambda_REV]

variances = []
for i in params:
    deviations = i - i.mean()
    sum_of_squared_dev = (deviations ** 2).sum()
    var = sum_of_squared_dev / (len(i) ** 2) 
    variances.append(var)


variances = pd.DataFrame(np.array(variances)).T
variances.columns = ['alpha', 'lambda_mkt', 'lambda_SMB', 'lambda_HML', 'lambda_REV']

#Shanken Adjustment
adjustment = (1 + (lambdas.mean().T @ inv(factors_3_rev.cov()) @ lambdas.mean()))
#Adjusted variances
variances = variances * adjustment

#t-tests
t_alpha = alpha.mean() / np.sqrt(variances['alpha'])
t_mkt = lambda_mkt.mean() / np.sqrt(variances['lambda_mkt'])
t_SMB = lambda_SMB.mean() / np.sqrt(variances['lambda_SMB'])
t_HML = lambda_HML.mean() / np.sqrt(variances['lambda_HML'])
t_REV = lambda_REV.mean() / np.sqrt(variances['lambda_REV'])
t_stats = pd.concat([t_alpha, t_mkt, t_SMB, t_HML, t_REV], axis=1)
t_stats.columns = ['t_alpha', 't_mkt', 't_SMB', 't_HML', 't_REV']
t_stats = np.round(t_stats,2)

print("Factor premia and pricing error for the 3 factor and Short Term Reversal model are:")
print()
print("Pricing error:", alpha.mean())
print()
print("Factor Premia:", np.round(lambdas.mean(),4))
print()
print(t_stats)


Factor premia and pricing error for the 3 factor and Short Term Reversal model are:

Pricing error: 0    1.13857
dtype: float64

Factor Premia: lambda_mkt   -0.5653
lambda_SMB    0.1429
lambda_HML    0.2861
lambda_REV   -0.7139
dtype: float64

   t_alpha  t_mkt  t_SMB  t_HML  t_REV
0     4.25  -1.77   1.16   2.45  -1.53
